In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.svm import SVR

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

import mlflow

In [2]:
mlflow.set_tracking_uri("https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow")

In [3]:
import dagshub
dagshub.init(repo_owner='gulamkibria775', repo_name='Uber-Delivery-prediction', mlflow=True)

Accessing as gulamkibria775

Initialized MLflow to track repo "gulamkibria775/Uber-Delivery-prediction"

Repository gulamkibria775/Uber-Delivery-prediction initialized!

In [4]:
# load the training and test data

train_data_path = "../data/processed/train.csv"
test_data_path = "../data/processed/test.csv"

train_df = pd.read_csv(train_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

test_df = pd.read_csv(test_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

train_df

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,187,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,194,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,180,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,197,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185,185.0,4
...,...,...,...,...,...,...,...,...
2016-02-29 22:45:00,15.0,9.0,11.0,11.0,29,12,12.0,0
2016-02-29 23:00:00,12.0,15.0,9.0,11.0,29,17,14.0,0
2016-02-29 23:15:00,17.0,12.0,15.0,9.0,29,15,14.0,0


In [5]:
# missing value in training data

train_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [6]:
# missing values in the test data

test_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [7]:
# make X_train and y_train

X_train = train_df.drop(columns=["total_pickups"])

y_train = train_df["total_pickups"]

In [8]:


# make X_test and y_test

X_test = test_df.drop(columns=["total_pickups"])

y_test = test_df["total_pickups"]



In [9]:
from sklearn import set_config

set_config(transform_output="pandas")

In [10]:


# encode the data

encoder = ColumnTransformer([
    ("ohe", OneHotEncoder(drop="first",sparse_output=False), ["region","day_of_week"])
], remainder="passthrough", n_jobs=-1,force_int_remainder_cols=False)



In [11]:
# encode the train and test data

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

In [12]:
import optuna
import tqdm 


In [13]:

mlflow.set_experiment("Model Selection")

2025/08/24 00:28:34 INFO mlflow.tracking.fluent: Experiment with name 'Model Selection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/2a268b19a95b44b4b63791d7dbbab215', creation_time=1755973714772, experiment_id='0', last_update_time=1755973714772, lifecycle_stage='active', name='Model Selection', tags={}>

In [14]:
def objective(trial):
    # start the child run
    with mlflow.start_run(nested=True) as child:
        
        # model name search space
        list_of_models = ["LR", "RF", "GBR", "XGBR"]
        model_name = trial.suggest_categorical("model_name", list_of_models)
    
        if model_name == "LR":
            model = LinearRegression()
    
        elif model_name == "RF":
            n_estimators_rf = trial.suggest_int("n_estimators_rf",10,100,step=10)
            max_depth_rf = trial.suggest_int("max_depth_rf",3,10)
            model = RandomForestRegressor(n_estimators=n_estimators_rf, 
                                          max_depth=max_depth_rf, 
                                          random_state=42, n_jobs=-1)
    
        elif model_name == "GBR":
            n_estimators_gb = trial.suggest_int("n_estimators_gb",10,100,step=10)
            learning_rate_gb = trial.suggest_float("learning_rate_gb",1e-4,1e-1, log=True)
            model = GradientBoostingRegressor(n_estimators=n_estimators_gb, 
                                              learning_rate=learning_rate_gb,
                                             random_state=42)
    
        elif model_name == "XGBR":
            n_estimators_xgb = trial.suggest_int("n_estimators_xgb",10,100,step=10)
            learning_rate_xgb = trial.suggest_float("learning_rate_xgb",1e-4,1e-1, log=True)
            max_depth_xgb = trial.suggest_int("max_depth_xgb",3,10)
            model = XGBRegressor(n_estimators=n_estimators_xgb,
                                learning_rate=learning_rate_xgb,
                                max_depth=max_depth_xgb)
    
        # log the model name
        mlflow.log_param("model_name",model_name)
        
        # log the model parameters
        mlflow.log_params(model.get_params())
        
        # fit on the data
        model.fit(X_train_encoded,y_train)
    
        # get the predictions
        y_pred = model.predict(X_test_encoded)
    
        # calculate the loss
        loss = mean_absolute_percentage_error(y_test, y_pred)
    
        # log the metric
        mlflow.log_metric("MAPE",loss)
        return loss

In [15]:


# optimize the objective function

with mlflow.start_run(run_name="best_model", nested=True) as parent:

    # create a study object
    study = optuna.create_study(study_name="model_selection", direction="minimize")
    # optimize the objective function
    study.optimize(func=objective, n_trials=50, n_jobs=-1)
    
    # log the best parameters
    mlflow.log_params(study.best_params)
    # log the best error value
    mlflow.log_metric("Best_MAPE", study.best_value)



[I 2025-08-24 00:29:11,308] A new study created in memory with name: model_selection


🏃 View run dashing-bass-17 at: https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow/#/experiments/0/runs/233c7449ee8e49dbbcdbb7bff3cc3c0f
🧪 View experiment at: https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow/#/experiments/0
🏃 View run tasteful-dog-334 at: https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow/#/experiments/0/runs/d0248de6ab124177bc1d8953b9848a62
🧪 View experiment at: https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow/#/experiments/0
🏃 View run beautiful-bee-945 at: https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow/#/experiments/0/runs/adde040fb5c34411ac577c58e1880b4c
🧪 View experiment at: https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow/#/experiments/0
🏃 View run luminous-yak-777 at: https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow/#/experiments/0/runs/eb0ade19c0694212b8fdf9b611c64444
🧪 View experiment at: https://dagshub.com/gulamkibria775/Uber-Delivery-pr

[I 2025-08-24 00:30:23,316] Trial 0 finished with value: 0.07934790285463365 and parameters: {'model_name': 'LR'}. Best is trial 0 with value: 0.07934790285463365.
[I 2025-08-24 00:30:24,323] Trial 2 finished with value: 0.07934790285463365 and parameters: {'model_name': 'LR'}. Best is trial 0 with value: 0.07934790285463365.


🏃 View run amazing-perch-4 at: https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow/#/experiments/0/runs/17ea4cde02914ca3ad0ad1c116babb7b
🧪 View experiment at: https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow/#/experiments/0


[I 2025-08-24 00:30:27,634] Trial 10 finished with value: 4.521983623504639 and parameters: {'model_name': 'XGBR', 'n_estimators_xgb': 40, 'learning_rate_xgb': 0.009465194087779225, 'max_depth_xgb': 10}. Best is trial 0 with value: 0.07934790285463365.


...

🏃 View run victorious-panda-161 at: https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow/#/experiments/0/runs/a6554094c5a549b9aba4a5ef0b5cc2bd
🧪 View experiment at: https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow/#/experiments/0


[I 2025-08-24 00:35:41,527] Trial 19 finished with value: 0.17696513204101527 and parameters: {'model_name': 'RF', 'n_estimators_rf': 50, 'max_depth_rf': 7}. Best is trial 0 with value: 0.07934790285463365.


🏃 View run upbeat-ray-90 at: https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow/#/experiments/0/runs/32ef00caef4d44d08616490faf516a9e
🧪 View experiment at: https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow/#/experiments/0


[I 2025-08-24 00:35:45,859] Trial 34 finished with value: 0.07934790285463365 and parameters: {'model_name': 'LR'}. Best is trial 0 with value: 0.07934790285463365.


🏃 View run best_model at: https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow/#/experiments/0/runs/945f00768b024398a0e3b537f77e470b
🧪 View experiment at: https://dagshub.com/gulamkibria775/Uber-Delivery-prediction.mlflow/#/experiments/0


In [16]:
# best value

study.best_value


0.07934790285463365

In [17]:


# best parameters

study.best_params



{'model_name': 'LR'}

In [18]:


# model value counts

study.trials_dataframe()['params_model_name'].value_counts()



params_model_name
LR      24
RF      10
GBR     10
XGBR     6
Name: count, dtype: int64

In [19]:
from optuna.visualization import (
    plot_optimization_history, 
    plot_parallel_coordinate, 
    plot_param_importances
)

In [20]:
plot_optimization_history(study)

In [21]:
plot_parallel_coordinate(study, params=["model_name"])

In [22]:
# train the linear regression model

lr = LinearRegression()

lr.fit(X_train_encoded, y_train)

# get predictions
y_pred_train = lr.predict(X_train_encoded) 
y_pred_test = lr.predict(X_test_encoded)

# loss

mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

print("The training error is ", mape_train)
print("The test error is ", mape_test)


The training error is  0.08778013304567367
The test error is  0.07934790285463365


In [23]:
lr.coef_

array([-2.33737604,  0.71512405, -0.55601505, -1.25311068, -3.20463231,
       -0.86685973, -2.79925402, -3.62516859,  0.41386463, -2.9376376 ,
       -1.97624678, -3.75050442,  0.51806283, -2.54033388, -2.43297463,
        0.47632075,  0.61254786, -4.7417372 , -2.03077217, -1.26960984,
       -4.03690273, -2.08863167, -1.0414428 ,  0.73561736, -0.99999442,
       -0.85944985, -2.43098478,  0.67112238,  0.57385071, -0.11719951,
       -0.28045898, -0.37180749, -0.5238324 , -0.4233113 , -0.34045774,
       -0.54170892, -0.36264553, -0.2493965 , -0.31905518,  2.4912456 ])

In [24]:
def tune_ridge(trial):
    # hyperparameter space
    alpha = trial.suggest_float("alpha",30,100)
    
    # make the model object
    ridge = Ridge(alpha=alpha, random_state=42)
    
    # train the model
    ridge.fit(X_train_encoded, y_train)
    
    # get predictions
    y_pred = ridge.predict(X_test_encoded)
    
    # calculate loss
    loss = mean_absolute_percentage_error(y_test, y_pred)

    return loss

In [25]:

# create study

study = optuna.create_study(study_name="tune_model", direction="minimize")

[I 2025-08-24 00:35:49,138] A new study created in memory with name: tune_model


In [26]:


# optimize

study.optimize(func=tune_ridge, n_trials=100, n_jobs=-1, show_progress_bar=True)



  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-08-24 00:35:49,438] Trial 9 finished with value: 0.07919196515138208 and parameters: {'alpha': 79.08336625130552}. Best is trial 9 with value: 0.07919196515138208.
[I 2025-08-24 00:35:49,469] Trial 1 finished with value: 0.07919934960348221 and parameters: {'alpha': 74.00442002421727}. Best is trial 9 with value: 0.07919196515138208.
[I 2025-08-24 00:35:49,485] Trial 8 finished with value: 0.07926791179891388 and parameters: {'alpha': 33.84645273060319}. Best is trial 9 with value: 0.07919196515138208.
[I 2025-08-24 00:35:49,494] Trial 13 finished with value: 0.07916536788600789 and parameters: {'alpha': 98.94054601658382}. Best is trial 13 with value: 0.07916536788600789.
[I 2025-08-24 00:35:49,503] Trial 6 finished with value: 0.07916849825204239 and parameters: {'alpha': 96.42961989212678}. Best is trial 13 with value: 0.07916536788600789.
[I 2025-08-24 00:35:49,504] Trial 11 finished with value: 0.07917130716326046 and parameters: {'alpha': 94.2222696765011}. Best is trial 

In [27]:
# best parameters

study.best_params

{'alpha': 99.99793791618914}

In [28]:
# best value

study.best_value

0.07916406809192203